<a href="https://colab.research.google.com/github/sonyongwoo/DeepLearning/blob/main/%EA%B9%80%EB%8B%AD%EB%82%A0%EA%B0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)

In [ ]:
#압축해제
cd "/content/drive/MyDrive/"

In [ ]:
#집파일로부터 폴더 생성

import os
import tensorflow as tf
import shutil

os.mkdir('/content/dataset')
os.mkdir('/content/dataset/fem')
os.mkdir('/content/dataset/male')

for i in os.listdir('/content/'):
    if 'fem' in i:
        shutil.move( '/content/' + i, '/content/dataset/fem/' + i )
    # if 'male' in i:
    #     shutil.move( '/content/' + i, '/content/dataset/male/' + i)
    

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/',
    image_size=(244,244),
    batch_size=32,
    subset='training',
    validation_split=0.2,
    seed=1234
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/',
    image_size=(244,244),
    batch_size=32,
    subset='validation',
    validation_split=0.2,
    seed=1234
)

#데이터 검증
print(train_ds)

#import matplotlib.pyplot as plt

#for i, 정답 in train_ds.take(1):
#    print(i)
#    print(정답)
#    plt.imshow(i[0].numpy().astype('uint8'))
#    plt.show()

In [ ]:

#VGG다운로드
vgg_dir = 'vgg/'
if not isdir(vgg_dir):
    os.makedirs(vgg_dir)
    
class DLProgress(tqdm):
    last_block = 0
    
    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num
        
if not isfile(vgg_dir + "vgg16.npy"):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc='VGG16 Params') as pbar:
        urlretrieve(
            'https://s3.amazonaws.com/content.udacity-data.com/nd101/vgg16.npy',
            vgg_dir + 'vgg16.npy',
            pbar.hook
        )
else:
    print("VGG16 already exists.")

In [ ]:
#Learning
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models 
from tensorflow.keras.applications import VGG16

base = VGG16(weights= 'imagenet', include_top=False, input_shape = (244,244,3))
base.trainable = False
for layer in base.layers:
  print('{} : {}'.format(layer, layer.trainable))
  
train_gen = ImageDataGenerator(horizontal_flip = True,
                              rotation_range = 60,
                              rescale = 1./255,
                              zoom_range = [0.7,1.5],
                              brightness_range = (0.7,1.0),
                              width_shift_range = 0.1,
                              height_shift_range = 0.1,
                              fill_mode='nearest',
                              validation_split=0.2) 
#val, gen generator
#VT_gen = ImageDataGenerator(rescale = 1./255)

batch_size = 32
#generator.flowfromdirectory
train_genorator = train_gen.flow_from_directory('/content/drive/MyDrive/chicken_sexing/training',
                                                 target_size = (244,244), class_mode = 'binary', shuffle=True, seed=1, color_mode='rgb', batch_size=batch_size, subset='training')
val_genorator = train_gen.flow_from_directory('/content/drive/MyDrive/chicken_sexing/training', 
                                               target_size = (244,244), class_mode = 'binary', shuffle=True, seed=7, color_mode='rgb', batch_size=batch_size, subset='validation')

#  test_genorator = VT_gen.flow_from_directory('/content/dataset',
#                                             target_size = (244,244), batch_size = batch_size, class_mode = 'binary', shuffle=True, seed=123, color_mode='rgb', batch_size=32)

# train_ds = tf.keras.preprocessing.image_dataset_from_directory(
#     '/content/dataset/',
#     image_size=(244,244),
#     batch_size=32,
#     subset='training',
#     validation_split=0.2,
#     seed=1234
# )

# val_ds = tf.keras.preprocessing.image_dataset_from_directory(
#     '/content/dataset/',
#     image_size=(244,244),
#     batch_size=32,
#     subset='validation',
#     validation_split=0.2,
#     seed=1234
# )

# def 전처리함수(i, 정답):
#     i = tf.cast( i/255.0, tf.float32 )
#     return i, 정답

# train_ds = train

model = models.Sequential()
model.add(base)
model.add(layers.BatchNormalization())
model.add(layers.Flatten())
model.add(layers.Dense(4096, activation = 'relu')) 
model.add(layers.Dropout(0.3))
model.add(layers.Dense(4096, activation = 'relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(1, activation = 'sigmoid'))

콜백함수 = tf.keras.callbacks.ModelCheckpoint(
    filepath='체크포인트/VGG16',
    save_weights_only=True,
    save_freq='epoch'
)

#sgd = tf.keras.optimizers.SGD(learning_rate= 1.e-4, momentum = 0.9)
model.compile(optimizer = "adam", loss = 'binary_crossentropy',metrics = ['accuracy'])

from tensorflow.keras.callbacks import TensorBoard
import time

tensorboard = TensorBoard( log_dir='logs/{}'.format( '첫모델' + str(int(time.time()) )) )

#EarlyStopping
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping( monitor='val_loss', patience=5, mode='min' )

epochs = 100

hist = model.fit(train_genorator, batch_size = batch_size, epochs= epochs , validation_data = val_genorator, callbacks=[콜백함수, tensorboard])

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs